# **Proyecto Módilo I: Web Scraping**

## **Diplomado en Ciencia de Datos, UNAM 2025**

#### **Autor: Mtro. José César Romero Galván**

Proyecto de Web Scraping sobre el número de iniciativas presentadas en el Congreso de la Ciudad de México en el año legislativo 2024 - 2025 de la III Legislatura.

### **Objetivos:**

Extracción: Nombre de la iniciativa, nombre del suscriptor o suscriptora, partido político, institución, asociación y URL

Salida: Iniciativas_CDMX_2024_2025.CSV



In [ ]:
# 1) Instalamos BeautifulSoap (Ya viene instalado en colab)

! pip install beautifulsoup4

# "!" se usa en entornos de shell (como Bash o Zsh) para indicar que el comando que sigue
# debe ejecutarse en la shell ignorando cualquier configuración de tu entorno virtual actual.

In [ ]:
# 2) Se hace la importación de las bibliotecas con las que vamos a trabajar

from bs4 import BeautifulSoup     # Parseo (extracción de datos) de HTML
import requests                   # Se hacen peticiones HTTP para descargar páginas web
import re                         # Se utiliza para crear expresiones regulares (buscar patrones en el texto)
import time                       # Sirve para hacer pausas entre las peticiones, esto sirve para evitar bloqueos

from urllib.parse import urljoin  # Sirve para construir URLs absolutas desde rutas relativas, es decir, ayuda a establecer
                                  # URLs para que requests pueda acceder a la páginación de las iniciativas del Congreso CDMX
import pandas as pd               # Manejo de tablas (DataFrames) y para la exportación de CSV de la tabla con los dato del web scrapping


In [ ]:
# 3) Configuración del Web Scraping

BASE = "https://ciudadana.congresocdmx.gob.mx"           # Dominio base del sitio web en donde se hará el web scraping
LIST_URL = f"{BASE}/Iniciativa/iniciativas"              # URl de la complementaria del sitio web donde se encuentran
                                                         # las iniciativas del Congreso de la Ciudad de México
                                                         # Se hace un f-string para poder insertar el valor {BASE}
OUT_CSV = "iniciativas_CDMX_2024_2025.csv"               # Nombre del archivo de salida en formato CSV
PAUSE = 0.4                                              # Pausa en segundos entre peticiones para evitar bloqueos del servidor

session = requests.Session()                             # Crea una sesión HTTP reutilizable
                                                         # Permite configurarlo "una vez" y  se aplica a todas las peticiones del scraping.
session.headers.update({"User-Agent": "Mozilla/5.0 (raspado academico cdmx)"}) # Define User-Agent "Amigable"
                                                                               # El "User-Agent" ayuda a que el servidor te reconozca como navegador “real”
                                                                               # Algunos sitios bloquean agentes desconocidos
req = session.get(LIST_URL, timeout=30)                  # Hacemos la solicitud para entrar a la página
req.raise_for_status()                                   # Revisa el código HTTP de la respuesta y si es un error (4xx o 5xx)
                                                         # lanza una excepción (requests.HTTPError) para evitar seguir trabajando con errores

print(f"Estado de la Solicitud: {req.status_code}")      # Checa el estado de la solitud hacía la página, 200 indica que el intento fue éxitoso.

Estado de la Solicitud: 200


In [ ]:
# 4) Web Scraping

# Definimos la primera función para encontrar todos los resultados de la paginación del sitio web

def get_total_pages(soap: BeautifulSoup) -> int:          # Define una función que recibe un objeto BeautifulSoup y
                                                          # devuelve un número entero (int) para detectar cuantas páginas hay (1 - 46)
                                                          # soap es el HTML de la página de listado ya parseado con BeautifulSoup
    text = soap.get_text(" ", strip=True)                 # Extrae todo el texto visible de la página
                                                          # get_text(" ", strip=True) convierte el HTML en una cadena “plana” y legible para regex
    m = re.search(r"Page\s+\d+\s+of\s+(\d+)", text, flags=re.I)  # Expresión regular que busca el patrón “Page 1 of X”
                                                                 # flags=re.I es un parámetro de re.search/re.compile que no distigue mayúsculas
                                                                 # de minúsculas al comparar, útil si queremos que nuestro patrón funcione en todos
                                                                 # los caso (page, Page, PAGE, pAgE, PaGe, ETC.)
    return int(m.group(1)) if m else 1                    # Devuelve X si existe; si no, asume 1 página para que el código no tenga error.

In [ ]:
# Definimos la segunda función para normalizar texto extraído del HTML (evitar “basura” como múltiples espacios, tabs o saltos de línea).

def _clean_text(x: str) -> str:                                 # Define una función que recibe un string y devuelve un string "limpio"
    if x is None:                                               # Si el valor de entrada es None (no hay texto)
        return None                                             # Devuelve None sin intentar procesarlo
    return re.sub(r"\s+", " ", x.strip())                       # x.strip(): quita espacios/saltos al inicio y al final
                                                                # re.sub(r"\s+", " ", ...): colapsa cualquier secuencia de espacios,
                                                                # tabs o saltos de la linea en un solo espacio

In [ ]:
# Definimos la tercera función para obtener información en los nodos como nombre y valor

def extract_nombre_from_detail(detail_html: str):               # Define una función llamada extract_nombre_from_detail.
                                                                # Recibe como parámetro detail_html (un string con HTML).
    """
    Extrae SOLO el valor de 'Nombre:' de la ficha. (Sin usar h1/h2 para no confundir con el promovente y evitar falsos positivos porque en muchas fichas esos encabezados traen el promovente, no el título de la iniciativa.
)
    Soporta: <dt>Nombre</dt><dd>VALOR</dd> y <b>Nombre:</b> VALOR

    Dos formatos principales:

Definición (<dt>/<dd>): patrón muy común en fichas. Se toma el <dd> que sigue al <dt>Nombre</dt>.

Inline (negritas u otros): <b>Nombre:</b> Valor. Se leen los next_siblings hasta topar un separador (<br>, <hr>) o un nuevo <dt>.
    """

    s = BeautifulSoup(detail_html, "html.parser")                        # Parsea el HTML en un árbol navegable
                                                                         # s es el objeto raíz que permite buscar nodos, recorrer hijos, etc.
    label = s.find(                                                      # Busca una etiqueta (<b>, <strong>, <span>, <label> o <dt>).
                                                                         # Filtra por el texto que empiece con "nombre".
                                                                             # label será la etiqueta que contiene el título del campo (“nombre”).
        lambda tag: tag.name in ("b", "strong", "span", "label", "dt")   # lambda tag: es una función anónima que recibe cada tag de HTML en donde se revisan 2 condiciones
                                                                         # 1) Que la etiqueta se llame <b>, <strong>, <span>, <label> o <dt>.
                                                                         # 2) Que el texto que contiene empiece con "nombre" (ignorando mayúsculas y espacios).

        and tag.get_text(strip=True).lower().startswith("nombre"))       # .get_text es un método de BeautifulSoup que extrae todo el contenido dentro de una etiqueta HTML, ignorando etiquetas hijas
                                                                         # Strip=True es un parámetro que quita espacios en blanco al inicio y al final del texto, además, colapsa espacios extra
                                                                         # .lower() es un método de strings de Python que convierte todo el texto a minúsculas, se usa para que la comparación
                                                                         # no dependa de mayúsculas o minúsculas
                                                                         # .startswith("nombre"): Es un método de strings en Python,
                                                                         # que pregunta si una cadena empieza con cierto prefijo
                                                                         # Significa: “¿el texto del nodo comienza con la palabra nombre?”

    if label:                                                            # Si se encontró la etiqueta del campo "nombre"
        if label.name =="dt":                                            # find_next("") es un método de BeautifulSoup que busca el siguiente nodo en el árbol HTML que cumpla con. el selector ("dd")
                                                                         # Caso 1: Formato de definición <dt>Nombre</dt><dd>Valor</dd>
            dd = label.find_next("dd")                                   # Toma el siguiente valor <dd> (El valor emparejado)
            if dd:
                val = _clean_text(dd.get_text(" ", strip=True))          # _clean_text(): función auxiliar que limpia espacios dobles, saltos de línea, etc.
                                                                         # Si encuentra el valor (val), lo devuelve.
                if val:
                    return val                                           # Devuelve el valor si exite éxito
        parts = []                                                       # Caso 2: Formato inline: <b>Nombre:</b> Valor (u otros tags)
        for sib in label.next_siblings:                                  # Recorre los hermanos que vienen después del label
            if getattr(sib, "name", None) in ("br", "hr", "dt"):         # Si aparece un separador o nuevo <dt>, cortamos
                break                                                    # hasattr() es una función incorporada de Python que responde:
                                                                         # ¿el objeto "x" tiene un atributo llamado attr? Si sí, devuelve True.
                                                                         # getattr() además de verificar, lo obtiene (o devuelve default si no existe).
            txt = sib.get_text(" ", strip=True) if hasattr(sib, "get_text") else str(sib)  # Sirve para obtener texto hermano
            txt = _clean_text(txt)                                       # Limpia los espacios y los saltos
            if txt:
                parts.append(txt)                                        # Acumula los fragmentos del valor
        lab_txt = _clean_text(label.get_text(" ", strip=True))           # Revisa si el label contiene "Nombre: VALOR"
        if lab_txt and ":" in lab_txt:                                   # Si el propio label trae "Nombre: Algo"
            after = lab_txt.split(":", 1)[1].strip()                     # Toma lo que viene despues de los dos puntos
            if after:
                parts.insert(0, after)                                   # Inserta primero para que quede antes que los siblings
        val = _clean_text(" ".join(parts)).strip(":") if parts else None # Une y limpia posible ":" residuales
        if val:
            return val                                                   # Devuelve el valor obtenido en formato inline
    full = s.get_text("\n", strip=True)                                # Fallback: toma TODO el texto plano de la ficha (con saltos)
    m= re.search(r"^\s*Nombre\s*:\s*(.+)$", full, flags=re.I | re.M)     # Busca una línea que empiece con "Nombre:"
    if m:
        return _clean_text(m.group(1))                                   # Devuelve el valor
    return None                                                          # Si no se encontró, devuelve el valor None

In [ ]:
# Definimos la cuarta función para  extraer información del enlace "Ver más" (nombre y partido político)

def pull_card_context(anchor):                                           # Extrae datos del "card" (lista) alrededor del enlace "Ver más"
                                                                         # anchor es un nodo del DOM (árbol HTML), en este caso tiene el <a> (Ver más)
    el = anchor                                                          # Comienza desde el enlace de "Ver más"
    block_text = ""                                                      # Inicializa contenedor de texto de tarjeta
    for _ in range(6):                                                   # Sube hasta el nivel 6 del árbol DOM
        if not el:
            break
        block_text =el.get_text("\n", strip=True)                        # Toma el texto del nodo actual
        if "Suscrita por" in block_text:                                 # Si contiene "Suscrita por", ya es el bloque deseado
            break
        el = el.parent                                                   # Si no, sigue subiendo al padre
    m = re.search(r"Suscrita por:\s*(.*?)\s*\|\s*(.*?)\s*\|", block_text, flags=re.I)   # Expresión regular que captura NOMBRE y PARTIDO/ROL
    if m:
        suscriptor_nombre = _clean_text(m.group(1))                      # Limpia y toma el primer grupo (nombre)
        suscriptor_partido_rol = _clean_text(m.group(2))                 # Limpia y toma el segundo grupo (partido/institución/rol)
    else:
        suscriptor_nombre = None                                         # Si no se encontró, deja None
        suscriptor_partido_rol = None
    return suscriptor_nombre, suscriptor_partido_rol                     # Devuelve una tupla con (nombre, partido/rol)

In [ ]:
# 5) Title case español con conectores en minúscula para mantener en minúscula ciertas palabras de enlace (artículos, preposiciones, conjunciones) salvo si son la primera o última palabra

# Definimos la primera funcion para formatear el texto a “Title Case” español (con stopwords en minúscula)

def word_title_text(s: str, keep_acronyms=None) -> str:                  # Formatea texto a “Title Case” español (con stopwords en minúscula)
  """
    Title case por palabra con 'stopwords' en minúscula (salvo si son la primera o la última palabra).
    - Reemplaza '_' por espacio y colapsa espacios.
    - Pasa todo a minúsculas y capitaliza palabra a palabra.
    - Mantiene en minúscula conectores/artículos/preps (o, y, e, u, de, del, de la, la, el, los, las, a, al, en, con, por, para, sin, sobre, entre, hacia, hasta, según, tras, como, que).
    - Preserva acrónimos dados en `keep_acronyms` (p. ej., {"CDMX","UNAM"}).
    """
  if pd.isna(s) or not isinstance(s, str):                             # pd.isna(s) (Pandas): devuelve True si s es NaN, None, NaT, o equivalente de pandas. Protege contra valores faltantes
                                                                       # instance(s, str): comprueba que s sea un str
    return s                                                           # Si es NaN o no es string, regresa tal cual
  stopwords = {                                                        # Se crea un set con los acrónimos que queremos preservar tal cual en minúsculas
      "a","al","ante","bajo","cabe","con","contra","de","del","desde","en","entre","hacia",
      "hasta","para","por","según","sin","so","sobre","tras","y","e","o","u","la","el","los",
      "las","un","una","unos","unas","que","como","lo","del","de","dela","dell","dela"}
  keep = set(keep_acronyms or [])                                      # Conjunto de acrónimos a preservar
  s2 = re.sub(r"_+", " ", s)                                           # Sustituye uno o varios _ contiguos por un único espacio
  s2 = re.sub(r"\s+", " ", s2.strip()).lower()                         # Reduce múltiples espacios a uno solo y pasa a minúsculas
  if not s2:                                                           # Si no hay texto, regresa tal cual
    return s2
  words = s2.split(" ")                                                # Divide en palabras
  n = len(words)                                                       # Número de palabras del objeto words que se guarda en el objeto n
  out = []                                                             # Acumulará palabras formateadas
  for i, w in enumerate(words):                                        # Itera cada palabra y su posición, es decir, se recorre cada palabra w con su índice i.
    word = w                                                           # Regla: si es la primera palabra (i==0) o la última (i==n-1)
                                                                       # o la palabra NO es una stopword, se capitaliza la primera letra: w[:1].upper() + w[1:].
                                                                       # Por defecto, deja la palabra en minúscula
    if i == 0 or i == n - 1 or w not in stopwords:                     # Capitaliza si es 1ª/última o no es stopword
                                                                       # Ej: "ley de tránsito" → ["Ley", "de", "Tránsito"].
                                                                       # Si la palabra es stopword y está en medio, se deja en minúscula.
      word = w[:1].upper() + w[1:]
    out.append(word)                                                   # Agrega la palabra resultante
  result = " ".join(out)                                               # Finalmente se reconstruye la cadena con " ".join(out).

  if keep:                                                             # Si hay acrónimos a preservar (p. ej., CDMX)
    tokens = result.split(" ")                                         # Vuelve a dividir por palabra
    tokens = [
        t if t.lower() not in {k.lower() for k in keep}
        else next(iter({k for k in keep if k.lower()==t.lower()}))
        for t in tokens]
    result = " ".join(tokens)
  return result                                                        # Devuelve el texto en “title case” español

In [ ]:
# Definimos la segunda función para renombrar las columns del archivo CSV que se va a exportar al final

def normalize_headers_with_mapping(df: pd.DataFrame) -> pd.DataFrame:  # Establece nombres finales de columnas
    """
    Renombra columnas al catálogo final y aplica word_title_text a cualquier otra columna que aparezca.
    """
    rename_map = {                                                     # Mapa de nombres internos a nombres finales
        "nombre_iniciativa": "Nombre de la Iniciativa",
        "suscriptor_nombre": "Nombre del Suscriptor o Suscriptora",
        "suscriptor_partido_rol": "Partido Político, Institución o Asociación",
        "url": "Url",
    }
    df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})      # Aplica renombrado donde aplique
    df.columns = [word_title_text(c, keep_acronyms={"CDMX","UNAM"}) for c in df.columns]  # Aplica “title case” español
    return df                                                                             # Devuelve el DataFrame con encabezados finales


In [ ]:
# Definimos la tercera función para terminar de formatear el archivo CSV que se exporta al final


def normalize_dataframe_content(df: pd.DataFrame, url_col_name: str = "Url") -> pd.DataFrame:  # Formatea contenido
    """
    Aplica word_title_text al contenido de columnas de texto (excepto Url).
    """
    df = df.copy()                                                                             # Trabaja sobre una copia
    for col in df.select_dtypes(include="object").columns:                                     # Recorre columnas de tipo texto
        if col == url_col_name:                                                                # Salta la columna Url (no tocar enlaces)
            continue
        df[col] = df[col].apply(lambda x: word_title_text(x, keep_acronyms={"CDMX","UNAM"}))   # Aplica formateo
    return df                                                                                  # Devuelve el DataFrame formateado

In [ ]:
# 6) Proceso principal
def main():                                               # Función principal del script
    r = session.get(LIST_URL, timeout=30)                 # Solicita la página de lista (desactiva verificación SSL)
    r.raise_for_status()                                  # Lanza error si la respuesta HTTP no fue exitosa
    soup = BeautifulSoup(r.text, "html.parser")           # Parsea el HTML de la lista
    total_pages = get_total_pages(soup)                   # Detecta cuántas páginas hay en total
    print(f"[INFO] Total de páginas detectadas: {total_pages}")  # Log informativo

    rows, seen_detail = [], set()                         # Inicializa acumulador de filas y set de detalles ya vistos

    for p in range(1, total_pages + 1):                   # Itera cada página del paginador
        url = LIST_URL if p == 1 else f"{BASE}/Iniciativa/iniciativas?page={p}"  # Arma URL de la página p
        print(f"[INFO] Página {p}/{total_pages}: {url}")  # Log de progreso
        resp = session.get(url, timeout=30)               # Descarga el HTML de la página p
        resp.raise_for_status()                           # Verifica éxito de la solicitud
        s = BeautifulSoup(resp.text, "html.parser")       # Parsea HTML de la página p

        anchors = s.find_all("a", string=re.compile(r"ver más", re.I))  # Busca todos los enlaces “Ver más”
        for a in anchors:                                 # Itera cada tarjeta/enlace encontrado
            href = a.get("href")                          # Obtiene href del enlace
            if not href:
                continue                                  # Si no hay href, salta
            detail_url = urljoin(BASE, href)              # Convierte href relativo en URL absoluta
            if detail_url in seen_detail:                 # Evita procesar el mismo detalle dos veces
                continue
            seen_detail.add(detail_url)                   # Marca el detalle como visto

            suscriptor_nombre, suscriptor_partido_rol = pull_card_context(a)  # Extrae datos del card (lista)

            d = session.get(detail_url, timeout=30)       # Solicita la página de detalle
            d.raise_for_status()                          # Verifica éxito
            nombre_iniciativa = extract_nombre_from_detail(d.text)  # Extrae “Nombre:” de la ficha

            rows.append({                                 # Agrega una fila al resultado
                "nombre_iniciativa": nombre_iniciativa,
                "suscriptor_nombre": suscriptor_nombre,
                "suscriptor_partido_rol": suscriptor_partido_rol,
                "url": detail_url,
            })
            time.sleep(PAUSE)                             # Pausa corta entre detalles
        time.sleep(PAUSE)                                 # Pausa corta entre páginas

    df = pd.DataFrame(rows)                               # Crea DataFrame con todas las filas extraídas

    df = normalize_headers_with_mapping(df)               # Aplica nombres finales de columnas (title case español)
    df = normalize_dataframe_content(df, url_col_name="Url")  # Formatea contenido (excepto Url)

    df.to_csv(OUT_CSV, index=False, encoding="utf-8")     # Exporta el DataFrame a CSV
    print(f"[OK] CSV generado: {OUT_CSV}  ({len(df)} filas)")  # Mensaje final con ruta y número de filas

if __name__ == "__main__":                                # Punto de entrada del script
    main()                                                # Ejecuta la función principal


[INFO] Total de páginas detectadas: 46
[INFO] Página 1/46: https://ciudadana.congresocdmx.gob.mx/Iniciativa/iniciativas


KeyboardInterrupt: 

### **Nota final:**

En este proyecto se aplicaron técnicas vistas en el módulo, como expresiones regulares para identificar y extraer patrones de texto (p. ej., “Page 1 of X” o “Nombre:”); funciones y funciones lambda para encapsular y limpiar la lógica de parsing; ciclos for in range para recorrer la paginación y los enlaces de cada página; estructuras if/else para controlar el flujo ante casos faltantes o variantes del HTML; y métodos de listas como `.append()`para ir acumulando los registros obtenidos.

Finalmente, se utilizó pandas para estructurar la información en un DataFrame y exportarla a CSV, garantizando encabezados y contenido con el formato solicitado.
